In [ ]:
!pip install roboflow
!pip install ultralytics
! pip install ruamel.yaml
! wget https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8m-seg.pt

from roboflow import Roboflow
import sys
import ruamel.yaml
from ultralytics import YOLO
from pathlib import Path
import os
from collections import defaultdict
import copy
import math
import shutil
import numpy as np
import psutil
import gc
import matplotlib.pyplot as plt
import locale
from IPython.display import clear_output
import ctypes
import ctypes.util
import torch



rf = Roboflow(api_key="tD9RSesaXxGqmwVD6eVZ")
project = rf.workspace("roboarm").project("feet-qevah")
dataset = project.version(10).download("yolov8")


%cd /kaggle/working/feet-10

gc.enable()

libc = ctypes.CDLL(ctypes.util.find_library('c'))
libc.malloc_trim(ctypes.c_int(0))

torch.set_num_threads(1)
os.environ["OMP_NUM_THREADS"] = "1"


In [ ]:
class YoloModel:
    def __init__(self, path_to_model: str, path_to_yaml: str,
                 train_perc: float, test_perc: float, val_perc: float,):
        """Инициализация переменных

        Args:
            path_to_model (str): путь до весов yolov8.pt
            path_to_yaml (str): путь до data.yaml файла датасета
            train_perc (float): доля тренировочных данных 
            test_perc (float): доля тестовых данных
            val_perc (float): доля валидационных данных
        """        
        self.path_to_model = path_to_model
        self.path_to_yaml = path_to_yaml
        self.train_path = Path('train') if os.path.exists(Path('train')) else None
        self.test_path = Path('test') if os.path.exists(Path('test')) else None
        self.val_path = Path('valid') if os.path.exists(Path('valid')) else None
        self.train_perc = train_perc
        self.test_perc = test_perc
        self.val_perc = val_perc
        
        self.ttvs_flag = 0
        self.tp_flag = 0
        
        self.ram_usage = []
        
        assert self.train_path != None, 'Директория train отсутствует'
        if (self.val_path == None):
            os.mkdir("valid", mode=0o777)
            os.mkdir("valid/images", mode=0o777)
            os.mkdir("valid/labels", mode=0o777)
            self.val_path = Path("valid")
        if (self.test_path == None):
            os.mkdir("test", mode=0o777)
            os.mkdir("test/images", mode=0o777)
            os.mkdir("test/labels", mode=0o777)
            self.test_path = Path("test")
    
    def get_ram_usage(self,):
        """Gets the current RAM usage of the system.

        Returns:
            float: RAM usage in GB.
        """
        return psutil.virtual_memory().used / 1e9

    def train(self, folder_name: str, iters: int) -> YOLO: 
        """Инициализация модели и обучение

        Args:
            folder_name (str): название директории с частью данных
            iters (int): кол-во интераций

        Returns:
            YOLO: экземпляр обученной модели
        """        
        # Корректируем data.yaml файл
        yaml = ruamel.yaml.YAML()
        # yaml.preserve_quotes = True
        with open('data.yaml', 'r+') as fp:
            data = yaml.load(fp)
            for elem in data:
                if elem == 'train':
                    data[elem] = './'+folder_name+'/train/images'
                elif elem == 'val':
                    data[elem] = './'+folder_name+'/valid/images'
                elif elem == 'test':
                    data[elem] = './temp/images'
            fp.truncate(0)
            fp.seek(0)
            yaml.dump(data, fp)
        self.ram_usage.append(self.get_ram_usage())
        model = YOLO(self.path_to_model)
        model.train(data=self.path_to_yaml, pretrained=self.path_to_model,
                    exist_ok=True, epochs=iters,)
        return model
    
    def retrain(self, folder_name: str, iters: int,) -> YOLO:
        """Обучение модели на всех предыдущих частях данных для профилакти просадки метрики

        Args:
            folder_name (str): название директории с частью данных
            iters (int): кол-во интераций

        Returns:
            YOLO: экземпляр обученной модели
        """        
        os.mkdir("retrain", mode=0o777)
        os.mkdir("retrain/train", mode=0o777)
        os.mkdir("retrain/valid", mode=0o777)
        os.mkdir("retrain/train/images", mode=0o777)
        os.mkdir("retrain/train/labels", mode=0o777)
        os.mkdir("retrain/valid/images", mode=0o777)
        os.mkdir("retrain/valid/labels", mode=0o777)

        # собираем список всех кусков данных до нашего folder_name
        folder_num = np.arange(1, int(folder_name.split('_')[-1]) + 1)
        source_pathes = [f"temp_{i}" for i in folder_num]

        # копируем все собранные куски данных в папку retrain
        for path in source_pathes:
            # собираем все файлы
            all_images_train = os.listdir(path+"/train/images")
            all_labels_train = os.listdir(path+"/train/labels")

            all_images_valid = os.listdir(path+"/valid/images")
            all_labels_valid = os.listdir(path+"/valid/labels")

            for image in all_images_train:
                shutil.copyfile(path+"/train/images/" + image,
                                "retrain/train/images/" + image)
            for label in all_labels_train:
                shutil.copyfile(path+"/train/labels/" + label,
                                "retrain/train/labels/" + label)
            for image in all_images_valid:
                shutil.copyfile(path+"/valid/images/" + image,
                                "retrain/valid/images/" + image)
            for label in all_labels_valid:
                shutil.copyfile(path+"/valid/labels/" + label,
                                "retrain/valid/labels/" + label)

        # Корректируем data.yaml файл
        yaml = ruamel.yaml.YAML()
        with open('data.yaml', 'r+') as fp:
            data = yaml.load(fp)
            for elem in data:
                if elem == 'train':
                    data[elem] = 'retrain/train/images'
                elif elem == 'val':
                    data[elem] = 'retrain/valid/images'
                elif elem == 'test':
                    data[elem] = './temp/images'
            fp.truncate(0)
            fp.seek(0)
            yaml.dump(data, fp)
        
        self.ram_usage.append(self.get_ram_usage())
        # load a pretrained model (recommended for training)
        model = YOLO(self.path_to_model)
        model.train(data=self.path_to_yaml, pretrained=self.path_to_model,
                    exist_ok=True, epochs=iters,)
        return model
    
    def test(self, model: YOLO):
        """Тестирование модели

        Args:
            model (YOLO): экземпляр обученной модели

        Returns:
            _type_: _description_
        """        
        metrics = model.val(data=self.path_to_yaml, split="test")
        return metrics
    
    def train_test_val_split(self, keep_perc: float):
        """Разделение изначального датасета на заданные доли train/test/val. Удаление 1-keep_perc доли данных 

        Args:
            keep_perc (float): доля данных, которую нужно оставить
        """        
        # создаем директории для объединения всех файлов
        os.mkdir("temp", mode=0o777)
        os.mkdir("temp/images", mode=0o777)
        os.mkdir("temp/labels", mode=0o777)
            
        source_pathes = (self.train_path / 'images', self.train_path / 'labels',\
                        self.test_path / 'images', self.test_path / 'labels',\
                        self.val_path / 'images', self.val_path / 'labels',)
        destination = Path('temp')

        for path in source_pathes:
            # собираем все файлы
            allfiles = os.listdir(path)
            # итерируем по всем файлам, чтобы переместить их в папку назначения
            sub_folder = path.name # images or labels
            for f in allfiles:
                src_path = os.path.join(path, f)
                dst_path = os.path.join(destination / sub_folder, f)
                os.rename(src_path, dst_path)
        total_num = len(allfiles)
        # Создаем словарь, где ключ - метка класса, значение - множество путей к label файлам данного класса
        classes = defaultdict(set)
        empty_count = 0
        for txt_path in Path("temp/labels").glob("*.txt"):
            with txt_path.open() as f:
                text = f.read()
                # Каждому объекту на изображении соотвествует одна строка, где первое значение - метка класса,
                # а остальное - координаты сегментации
                for obj in text.split('\n'):
                    if len(obj) > 0:
                        classes[obj.split()[0]].add(txt_path)
                    else:
                        print(f"Пустой файл: {txt_path}")
                        empty_count += 1
        print(f"Кол-во пустых файлов - {empty_count}")
        # Оставляем указанный процент данных
        for cls, pathes in zip(list(classes.keys()), list(classes.values())):
            num_files = len(pathes)
            num_to_del = num_files*(1-keep_perc)
            for i, file_path in enumerate(pathes.copy()):
                if i+1 >= num_to_del:
                    break
                f = file_path.name.split('.')[:-1]
                f.append('jpg')
                try:
                    Path("temp",'images',".".join(f)).unlink()
                    file_path.unlink()
                except OSError as e:
                    # Файл не найден (скорее всего имеет несколько классов и был уже перемещен)
                    pass
                classes[cls].remove(file_path)

        # Распределяем данные по директориям train, test, val с учетом указанных пропорций
        class_copy = copy.deepcopy(classes)
        for cls, pathes in zip(list(classes.keys()), list(classes.values())):
            num_files = len(class_copy[cls])
            num_to_mv_train = int(num_files * self.train_perc)
            num_to_mv_test = int(num_files * self.test_perc)
            num_to_mv_val = int(num_files * self.val_perc)
            # print(num_files, num_to_mv, len(pathes))
            temp_dict_name = "train"
            for i, file_path in enumerate(pathes.copy()):
                if i+1 > num_to_mv_train and i+1 < num_to_mv_val + num_to_mv_train:
                    temp_dict_name = "valid"
                elif i+1 > num_to_mv_val + num_to_mv_train:
                    temp_dict_name = "test"
                f = file_path.name.split('.')[:-1]
                f.append('jpg')
                try:
                    os.replace(Path("temp","images",".".join(f)), Path(temp_dict_name,"images").joinpath(Path("temp","images",".".join(f)).name))
                    os.replace(file_path, Path(temp_dict_name,"labels").joinpath(file_path.name))
                except OSError as e:
                    # Файл не найден (скорее всего имеет несколько классов и был уже перемещен)
                    pass
                classes[cls].remove(file_path)
        shutil.rmtree("temp")

    def take_piece(self, piece_perc: float):
        """Разделение исходного датасета на части равные доле piece_perc по директориям temp_{n}, n = {1..1/piece_perc}

        Args:
            piece_perc (float): доля части данных, на которые нужно поделить датасет
        """        
        # создаем директории для объединения всех файлов
        os.mkdir("temp", mode=0o777)
        os.mkdir("temp/images", mode=0o777)
        os.mkdir("temp/labels", mode=0o777)

        source_pathes = (self.train_path / 'images', self.train_path / 'labels',
                        self.test_path / 'images', self.test_path / 'labels',
                        self.val_path / 'images', self.val_path / 'labels',)
        destination = Path('temp')

        for path in source_pathes:
            # собираем все файлы
            allfiles = os.listdir(path)
            # итерируем по всем файлам, чтобы переместить их в папку назначения
            sub_folder = path.name  # images or labels
            for f in allfiles:
                src_path = os.path.join(path, f)
                dst_path = os.path.join(destination / sub_folder, f)
                os.rename(src_path, dst_path)
        total_num = len(allfiles)
        # Создаем словарь, где ключ - метка класса, значение - множество путей к label файлам данного класса
        classes = defaultdict(set)
        empty_count = 0
        for txt_path in Path("temp/labels").glob("*.txt"):
            with txt_path.open() as f:
                text = f.read()
                # Каждому объекту на изображении соотвествует одна строка, где первое значение - метка класса,
                # а остальное - координаты сегментации
                for obj in text.split('\n'):
                    if len(obj) > 0:
                        classes[obj.split()[0]].add(txt_path)
                    else:
                        print(f"Пустой файл: {txt_path}")
                        empty_count += 1
        print(f"Кол-во пустых файлов - {empty_count}")
        self.num_folders = 1 / piece_perc
        for folder in range(int(self.num_folders)):
            os.mkdir(f"temp_{folder+1}", mode=0o777)

            os.mkdir(f"temp_{folder+1}/train", mode=0o777)
            os.mkdir(f"temp_{folder+1}/valid", mode=0o777)

            os.mkdir(f"temp_{folder+1}/train/labels", mode=0o777)
            os.mkdir(f"temp_{folder+1}/train/images", mode=0o777)
            os.mkdir(f"temp_{folder+1}/valid/labels", mode=0o777)
            os.mkdir(f"temp_{folder+1}/valid/images", mode=0o777)
        # Распределяем данные по директориям train, test, val с учетом указанных пропорций
        class_copy = copy.deepcopy(classes)
        for folder in range(int(self.num_folders)):
            folder_name = f"temp_{folder+1}"
            for cls, pathes in zip(list(classes.keys()), list(classes.values())):
                num_files = len(class_copy[cls])
                num_to_mv_train = int(num_files * self.train_perc * piece_perc)
                num_to_mv_val = int(num_files * self.val_perc * piece_perc)
                # print(num_files, num_to_mv, len(pathes))
                temp_dict_name = "train"
                for i, file_path in enumerate(pathes.copy()):
                    if i+1 > num_to_mv_train and i+1 < num_to_mv_val + num_to_mv_train:
                        temp_dict_name = "valid"
                    if i+1 >= num_to_mv_val + num_to_mv_train + 3:  # останавливаем только если это последняя папка
                        break
                    f = file_path.name.split('.')[:-1]
                    f.append('jpg')
                    try:
                        os.replace(Path("temp", "images", ".".join(f)), Path(
                            folder_name, temp_dict_name, "images").joinpath(Path("temp", "images", ".".join(f)).name))
                        os.replace(file_path, Path(
                            folder_name, temp_dict_name, "labels").joinpath(file_path.name))
                    except OSError as e:
                        # Файл не найден (скорее всего имеет несколько классов и был уже перемещен)
                        pass
                    classes[cls].remove(file_path)
    
    def plot_result(self, result_dict: dict, color_dict: dict):
        '''Функция для отрисовки графиков зависимости метрик от размера обучающей выборки

        Args:
            result_dict (dict): словарь с метриками {доля данных: массив метрик}
            color_dict (dict): словарь с индикаторами повторного обучения
        '''
        metrics_names = ("mAP","mAP50", "mAP75")
        for i, metric in enumerate(metrics_names):
            plt.scatter(list(result_dict.keys()), [j[i] for j in result_dict.values()], color=list(color_dict.values()), zorder=1)
            plt.plot(list(result_dict.keys()), [j[i] for j in result_dict.values()], linestyle='-')
            plt.ylabel(metric)
            plt.xlabel("Keep percent (%)")
            plt.show()
        
    def plot_ram_usage(self,):
        plt.plot(self.ram_usage, marker='o', linestyle='-', color='b')
        plt.xlabel('Iteration')
        plt.ylabel('RAM Usage (GB)')
        plt.title('RAM Usage During Training')
        plt.show()
    
    def increm_learning(self, keep_perc: float, piece_perc: float, iters: int):
        os.environ['WANDB_DISABLED'] = 'true'
        # делим датасет на тренировочную/тестовую/валидационную выборку
        if self.ttvs_flag == 0:
            self.train_test_val_split(keep_perc)
            self.ttvs = 1
        if self.tp_flag == 0:
            self.take_piece(piece_perc)
            self.tp = 1
        # путь к изначальным весам yolov8 для повторного обучения при понижении метрики
        native_path_to_model = self.path_to_model
        # словарь с метриками {доля данных: массив метрик}
        result_dict = {k_p:[] for k_p in range(int(self.num_folders))}
        # словарь с индикаторами повторного обучения
        color_dict = defaultdict(str)
        # переменная для отслеживания максимального map в целях профилактики просадок метрики
        max_map = 0
        
        # Инкрементальное обучение 
        for folder in range(int(self.num_folders)):
            libc = ctypes.CDLL(ctypes.util.find_library('c'))
            libc.malloc_trim(ctypes.c_int(0))

            torch.set_num_threads(1)
            folder_name = f"temp_{folder+1}"
            # дообучаем модель
            model = self.train(folder_name, iters)
            # тестируем модель
            metrics = self.test(model)
            # проверяем, что метрика улучшается
            if metrics.seg.map > max_map:
                max_map = metrics.seg.map
                # заносим метрики в словарь
                result_dict[folder].append(metrics.seg.map)
                result_dict[folder].append(metrics.seg.map50)
                result_dict[folder].append(metrics.seg.map75)
                color_dict[folder] = "green"
                clear_output(wait=True)
                del(model)
                del(metrics)
                gc.collect()
                self.path_to_model = "./runs/segment/train/weights/last.pt"
            else:
                self.path_to_model = native_path_to_model
                # дообучаем модель
                model = self.retrain(folder_name, iters)
                # тестируем модель
                metrics = self.test(model)
                if metrics.seg.map > max_map:
                    max_map = metrics.seg.map
                # заносим метрики в словарь
                result_dict[folder].append(metrics.seg.map)
                result_dict[folder].append(metrics.seg.map50)
                result_dict[folder].append(metrics.seg.map75)
                color_dict[folder] = "red"
                clear_output(wait=True)
                del(model)
                del(metrics)
                gc.collect()
                self.path_to_model = "./runs/segment/train/weights/last.pt"
                shutil.rmtree("retrain")
                
        print(f"Итоговый результат: \n {result_dict}")
        self.plot_result(result_dict, color_dict)

In [ ]:
locale.getpreferredencoding = lambda: "UTF-8"

PATH_TO_MODEL = "/kaggle/working/yolov8m-seg.pt"
PATH_TO_YAML = "/kaggle/working/feet-10/data.yaml"
TRAIN_PERC = 0.8
TEST_PERC = 0.1
VAL_PERC = 0.1
KEEP_PERC = 1.0
PIECE_PERC = 0.1
ITERS = 5

exp_1 = YoloModel(PATH_TO_MODEL, PATH_TO_YAML, TRAIN_PERC, TEST_PERC, VAL_PERC)
exp_1.increm_learning(KEEP_PERC, PIECE_PERC, ITERS)

In [ ]:
num_folders = 1 / PIECE_PERC
for folder in range(int(num_folders)):
    dir_path = f"temp_{folder+1}/train/labels"
    print(dir_path, len([entry for entry in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, entry))]))
    dir_path = f"temp_{folder+1}/train/images"
    print(dir_path, len([entry for entry in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, entry))]))
    dir_path = f"temp_{folder+1}/valid/labels"
    print(dir_path, len([entry for entry in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, entry))]))
    dir_path = f"temp_{folder+1}/valid/images"
    print(dir_path, len([entry for entry in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, entry))]), '\n')
    
